# Valid dates

### The GEFS data has missing dates. This returns me a list of dates that are present in the GEFS data to avoid errors.

In [68]:
import os
import pandas as pd

def generate_labeled_dates(
    nc_dir: str,
    tornado_csv: str,
    date_lower: str = '2001-01-01',
    date_upper: str = '2019-12-31',
    exclusion_dates: list = None,
    label0_step: int = 20,
    label1_step: int = 2,
    output_csv: str = 'all_labeled_dates.csv'
) -> pd.DataFrame:
    """
    Create and save a balanced set of dates labeled 0/1/2 for no/weak/strong tornado activity.

    Parameters
    ----------
    nc_dir : str
        Directory of NetCDF files named 'YYYYMMDDHHMM_daily_preproc_ml.nc'.
    tornado_csv : str
        CSV with tornado records ('slat','slon','mag','date').
    date_lower, date_upper : str
        Inclusive date range in 'YYYY-MM-DD'.
    exclusion_dates : list of str
        Dates to drop, default ['2011-04-27','2023-03-31','1974-04-04'].
    label0_step, label1_step : int
        Sampling intervals for classes 0 and 1.
    output_csv : str
        Path to write the labeled CSV.

    Returns
    -------
    pd.DataFrame
        ['date','classification'] for classes 0,1,2.
    """
    # 1) dates with model data
    files = [f for f in os.listdir(nc_dir) if f.endswith('_daily_preproc_ml.nc')]
    available = {
        pd.to_datetime(f.split('_')[0], format='%Y%m%d%H%M').date() for f in files
    }

    # 2) load & filter tornado data
    tor = pd.read_csv(tornado_csv)
    tor['day'] = pd.to_datetime(tor['date'], errors='coerce').dt.date
    low, high = pd.to_datetime(date_lower).date(), pd.to_datetime(date_upper).date()
    tor = tor[(tor['slat'].between(30,36)) & (tor['slon'].between(-93,-80)) &
              (tor['day'].between(low, high))]

    # 3) label days with tornadoes
    label1, label2 = [], []
    for day, grp in tor.groupby('day'):
        if day not in available: continue
        if grp['mag'].ge(2).any(): label2.append(day)
        elif grp['mag'].le(1).all(): label1.append(day)

    # 4) label no-tornado days
    all_days = pd.date_range(low, high).date
    present = set(tor['day'])
    label0 = [d for d in all_days if d in available and d not in present]

    # 5) exclude dates
    excl = {pd.to_datetime(d).date() for d in (exclusion_dates or
            ['2011-04-27','2023-03-31','1974-04-04'])}
    label0 = [d for d in label0 if d not in excl]
    label1 = [d for d in label1 if d not in excl]
    label2 = [d for d in label2 if d not in excl]

    # 6) subsample
    samp0 = sorted(label0)[::label0_step]
    samp1 = sorted(label1)[::label1_step]
    samp2 = sorted(label2)

    # 7) build & save DataFrame
    def df_from(dates, cls):
        return pd.DataFrame({'date':[d.isoformat() for d in dates],
                             'classification':cls})

    out = pd.concat([df_from(samp0,0), df_from(samp1,1), df_from(samp2,2)],
                    ignore_index=True)
    out.to_csv(output_csv, index=False)

    print(f"Samples: 0→{len(samp0)}, 1→{len(samp1)}, 2→{len(samp2)}")
    return out

labeled_df = generate_labeled_dates(
    nc_dir='/home/scratch/ahaberlie1/gefs_reforecast_preproc/',
    tornado_csv='/home/z1995995/data_science_for_the_geosciences/495/1950-2023_actual_tornadoes.csv',
    output_csv='all_labeled_dates.csv'
)
print(labeled_df.head())


No-tornado days sampled: 259
Weak-tornado days sampled: 256
Strong-tornado days sampled: 197


# Event Extraction

In [72]:
import os
import numpy as np
import pandas as pd
import xarray as xr

def generate_event_metrics(
    tornado_csv: str,
    labeled_dates_csv: str,
    reforecast_dir: str,
    output_csv: str,
    init_index: int = 4
) -> None:
    """
    Generate and append tornado event metrics from GEFS reforecast datasets.

    Parameters
    ----------
    tornado_csv : str
        Path to the CSV file containing tornado observations with columns
        'slat', 'slon', 'mag', and 'date'.
    labeled_dates_csv : str
        Path to the CSV file containing dates to process, with a 'date'
        column formatted as YYYY-MM-DD.
    reforecast_dir : str
        Directory containing preprocessed GEFS reforecast NetCDF files named
        as YYYYMMDD1200_daily_preproc_ml.nc.
    output_csv : str
        Path to the output CSV file where metrics will be appended.
    init_index : int, optional
        Initialization index to select along the 'init' dimension
        (default is 4).
    """
    tor_df = pd.read_csv(tornado_csv)
    label_ds = pd.read_csv(labeled_dates_csv)

    for date in label_ds['date']:
        date_rfm = date.replace('-', '')
        ds_path = os.path.join(reforecast_dir, f"{date_rfm}1200_daily_preproc_ml.nc")
        ds = xr.open_dataset(ds_path)

        # subset tornado reports to Dixie Alley
        tor_dixie = tor_df[
            (tor_df['slat'] >= 30) & (tor_df['slat'] <= 36) &
            (tor_df['slon'] >= -93) & (tor_df['slon'] <= -80)
        ]
        sigtor_dixie = tor_dixie[tor_dixie['mag'] >= 2]
        tor_day_dixie = tor_dixie[(tor_dixie['mag'] >= 0) & (tor_dixie['mag'] < 2)]

        # subset dataset to Dixie Alley region
        ds = ds.where(
            (ds.lat >= 30) & (ds.lat <= 36) &
            (ds.lon >= -93) & (ds.lon <= -80),
            drop=True
        )

        # assign class label
        if date in sigtor_dixie['date'].values:
            ds['class'] = 2
        elif date in tor_day_dixie['date'].values:
            ds['class'] = 1
        else:
            ds['class'] = 0

        # drop unused variables
        ds = ds.drop_vars([
            'prec', 'ship', 'z500', 'lapse57', 'mucape', 'frzlvl', 'ulcape', 'mlcape'
        ])

        # select the requested initialization slice
        ds = ds.isel(init=init_index)

        # compute metrics
        metrics = {
            "date": date,
            # sums
            "u10_sum":     float(ds.u10.max(dim="time").sum()),
            "v10_sum":     float(ds.v10.max(dim="time").sum()),
            "t2m_sum":     float(ds.t2m.max(dim="time").sum()),
            "td2m_sum":    float(ds.td2m.max(dim="time").sum()),
            "cprec_sum":   float(ds.cprec.max(dim="time").sum()),
            "srh500m_sum": float(ds.srh500m.max(dim="time").sum()),
            "srh1km_sum":  float(ds.srh1km.max(dim="time").sum()),
            "srh3km_sum":  float(ds.srh3km.max(dim="time").sum()),
            "scp_sum":     float(ds.scp.max(dim="time").sum()),
            "stp_sum":     float(ds.stp.max(dim="time").sum()),
            "vtp_sum":     float(ds.vtp.max(dim="time").sum()),
            "lcl_sum":     float(ds.lcl.max(dim="time").sum()),
            "shear6km_sum":float(ds.shear6km.max(dim="time").sum()),
            "mslp_sum":    float(ds.mslp.max(dim="time").sum()),
            "pwat_sum":    float(ds.pwat.max(dim="time").sum()),
            "sbcape_sum":  float(ds.sbcape_gfs.max(dim="time").sum()),
            "sbcin_sum":   float(ds.sbcin_gfs.max(dim="time").sum()),

            # medians
            "u10_median":     float(ds.u10.max(dim="time").median()),
            "v10_median":     float(ds.v10.max(dim="time").median()),
            "t2m_median":     float(ds.t2m.max(dim="time").median()),
            "td2m_median":    float(ds.td2m.max(dim="time").median()),
            "cprec_median":   float(ds.cprec.max(dim="time").median()),
            "srh500m_median": float(ds.srh500m.max(dim="time").median()),
            "srh1km_median":  float(ds.srh1km.max(dim="time").median()),
            "srh3km_median":  float(ds.srh3km.max(dim="time").median()),
            "scp_median":     float(ds.scp.max(dim="time").median()),
            "stp_median":     float(ds.stp.max(dim="time").median()),
            "vtp_median":     float(ds.vtp.max(dim="time").median()),
            "lcl_median":     float(ds.lcl.max(dim="time").median()),
            "shear6km_median":float(ds.shear6km.max(dim="time").median()),
            "mslp_median":    float(ds.mslp.max(dim="time").median()),
            "pwat_median":    float(ds.pwat.max(dim="time").median()),
            "sbcape_median":  float(ds.sbcape_gfs.max(dim="time").median()),
            "sbcin_median":   float(ds.sbcin_gfs.max(dim="time").median()),

            # max values
            "u10_max":     float(ds.u10.max(dim="time").max()),
            "v10_max":     float(ds.v10.max(dim="time").max()),
            "t2m_max":     float(ds.t2m.max(dim="time").max()),
            "td2m_max":    float(ds.td2m.max(dim="time").max()),
            "cprec_max":   float(ds.cprec.max(dim="time").max()),
            "srh500m_max": float(ds.srh500m.max(dim="time").max()),
            "srh1km_max":  float(ds.srh1km.max(dim="time").max()),
            "srh3km_max":  float(ds.srh3km.max(dim="time").max()),
            "scp_max":     float(ds.scp.max(dim="time").max()),
            "stp_max":     float(ds.stp.max(dim="time").max()),
            "vtp_max":     float(ds.vtp.max(dim="time").max()),
            "lcl_max":     float(ds.lcl.max(dim="time").max()),
            "shear6km_max":float(ds.shear6km.max(dim="time").max()),
            "mslp_max":    float(ds.mslp.max(dim="time").max()),
            "pwat_max":    float(ds.pwat.max(dim="time").max()),
            "sbcape_max":  float(ds.sbcape_gfs.max(dim="time").max()),
            "sbcin_max":   float(ds.sbcin_gfs.max(dim="time").max()),

            "classification": int(ds['class'])
        }

        metric_df = pd.DataFrame([metrics])
        write_header = not os.path.exists(output_csv)
        metric_df.to_csv(
            output_csv,
            mode='a',
            header=write_header,
            index=False
        )
        print(f"Successfully added {date}, classification {int(ds['class'])}")

    print("Finished processing all dates.")

if __name__ == "__main__":
    generate_event_metrics(
        tornado_csv="/home/z1995995/data_science_for_the_geosciences/495/1950-2023_actual_tornadoes.csv",
        labeled_dates_csv="all_labeled_dates.csv",
        reforecast_dir="/home/scratch/ahaberlie1/gefs_reforecast_preproc",
        output_csv="/home/scratch/BWEART2/tor_RF_event_values_shorts.csv",
        init_index=4
    )


Succesfully Added 2001-01-01, with classification 0
Succesfully Added 2001-01-26, with classification 0
Succesfully Added 2001-02-21, with classification 0
Succesfully Added 2001-03-22, with classification 0
Succesfully Added 2001-04-15, with classification 0
Succesfully Added 2001-05-09, with classification 0
Succesfully Added 2001-06-14, with classification 0
Succesfully Added 2001-07-11, with classification 0
Succesfully Added 2001-08-03, with classification 0
Succesfully Added 2001-08-27, with classification 0
Succesfully Added 2001-09-23, with classification 0
Succesfully Added 2001-10-19, with classification 0
Succesfully Added 2001-11-12, with classification 0
Succesfully Added 2001-12-15, with classification 0
Succesfully Added 2002-01-10, with classification 0
Succesfully Added 2002-02-03, with classification 0
Succesfully Added 2002-02-27, with classification 0
Succesfully Added 2002-03-25, with classification 0
Succesfully Added 2002-04-19, with classification 0
Succesfully 

In [ ]:
merged_ds = pd.concat([df1, df2,df0])
merged_ds.to_csv('labeled_dates.csv')

In [ ]:
tor_df = pd.read_csv("/home/z1995995/data_science_for_the_geosciences/495/1950-2023_actual_tornadoes.csv")


In [ ]:
x = xr.open_dataset('/home/scratch/ahaberlie1/gefs_reforecast_preproc/200212301200_daily_preproc_ml.nc')

In [ ]:
available_dates

set()

In [ ]:
x = pd.read_csv('/home/scratch/BWEART2/tor_RF_event_valuess.csv'

,date,u10_sum,v10_sum,t2m_sum,td2m_sum,cprec_sum,srh500m_sum,srh1km_sum,srh3km_sum,scp_sum,...,scp_median,stp_median,vtp_median,lcl_median,shear6km_median,mslp_median,pwat_median,sbcape_median,sbcin_median,classification
0,2001-03-12,1658.5000,3126.8125,53229.4375,30633.6250,2699.447510,242782.3750,470657.7500,9.033891e+05,582.496094,...,0.002930,-0.000000,-0.0,857.1875,27.0000,1021.6250,20.0000,6.0,0.0625,1
1,2001-06-18,4309.3750,5492.0625,116152.9375,79823.4375,9155.352539,98686.5625,188018.6875,3.083128e+05,1044.048828,...,0.000000,0.000000,-0.0,1432.7500,10.0000,1018.5000,41.6875,1290.0,-0.2500,1
2,2001-10-11,3332.1250,10315.1250,103223.1875,74876.1250,4459.046875,243918.5625,329878.5625,4.414451e+05,2553.470703,...,0.130859,0.000000,-0.0,1018.1875,13.0000,1020.7500,39.6875,1091.0,-0.0625,1
3,2001-12-07,10383.0000,7104.1875,68248.2500,47934.9375,1929.285522,299024.0000,445511.4375,7.337777e+05,185.726562,...,0.006836,0.000977,0.0,927.4375,27.0000,1023.8750,28.5000,6.0,0.1875,1
4,2002-10-12,-1711.0625,3175.0000,95548.0000,66360.8125,4568.133789,156487.7500,271566.2500,4.016329e+05,691.721680,...,0.004883,0.000000,-0.0,1044.7500,13.0625,1021.3125,35.6250,98.0,0.3125,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
145,2017-12-01,1239.8125,-2166.3750,54707.8125,18284.8750,246.613403,124709.7500,247255.1250,5.299416e+05,52.013672,...,0.000000,0.000000,0.0,1387.2500,23.3125,1026.5625,11.5000,0.0,0.2500,0
146,2018-12-13,5490.3125,12453.2500,49544.0000,40483.3750,2597.056152,704249.1875,983862.6250,1.322796e+06,673.649414,...,0.034180,0.000977,-0.0,352.4375,27.0000,1017.4375,25.3750,2.0,0.0625,0
147,2019-02-01,-1066.0000,-478.1875,18524.6875,-301.0000,641.876648,362393.1250,662622.3125,1.082120e+06,84.563477,...,0.000000,-0.000000,0.0,794.9375,27.0000,1031.1875,16.1250,0.0,0.2500,0
148,2019-03-10,11960.7500,11954.9375,61323.6875,47689.1875,8396.304688,452903.0000,754821.6875,1.159126e+06,1064.243164,...,0.081055,0.002930,0.0,614.6875,27.0000,1018.1875,31.6875,12.0,0.0625,0


In [46]:
import sklearn


x = pd.read_csv('/home/scratch/BWEART2/tor_RF_event_values_all.csv')


,date,u10_sum,v10_sum,t2m_sum,td2m_sum,cprec_sum,srh500m_sum,srh1km_sum,srh3km_sum,scp_sum,...,scp_median,stp_median,vtp_median,lcl_median,shear6km_median,mslp_median,pwat_median,sbcape_median,sbcin_median,classification
0,2001-01-01,3089.0000,1629.6250,22348.6250,6275.7500,877.676453,381578.5000,629752.3125,1.067620e+06,128.050781,...,0.000000,0.000000,-0.0,624.3750,27.0000,1028.1875,18.3125,0.0,0.1875,0
1,2001-01-03,8393.8750,4435.5000,16879.4375,-10549.2500,77.194946,363948.5625,528811.0000,8.707932e+05,29.475586,...,0.000000,0.000000,0.0,1157.3750,27.0000,1029.1875,10.1250,0.0,0.1250,0
2,2001-01-04,16911.5625,4798.5625,19100.6875,-38.3125,435.279785,423530.6250,644313.8125,1.119882e+06,59.402344,...,0.000000,0.000000,0.0,754.5625,27.0000,1024.1250,9.8125,3.0,0.1875,0
3,2001-01-05,16051.3125,5553.1250,10161.3750,-13502.0625,147.988342,481445.5625,656719.8125,1.023956e+06,18.549805,...,0.000000,-0.000000,-0.0,1035.3750,27.0000,1023.5625,9.1875,0.0,0.2500,0
4,2001-01-06,11679.5625,5083.6875,16171.8125,-4193.5625,389.186462,440460.6250,631692.6875,1.022756e+06,57.415039,...,0.000000,-0.000000,-0.0,958.6250,27.0000,1023.6875,10.5000,0.0,0.1875,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5872,2019-04-18,8819.5000,21023.0000,88414.3750,64963.8750,8550.261719,571568.1250,793386.6250,1.082325e+06,5221.807617,...,0.644531,0.088867,-0.0,921.0625,27.0000,1011.0625,37.1875,327.0,0.0000,2
5873,2019-04-19,18047.0625,15793.2500,79430.8750,54960.8750,7135.784668,398923.9375,582328.8750,9.750844e+05,3545.743164,...,0.265625,0.025391,-0.0,1003.9375,27.0000,1012.0625,31.8125,111.0,0.0000,2
5874,2019-04-25,9165.1250,12443.8125,89505.6875,63057.9375,5615.302734,321854.3125,417810.0000,6.222134e+05,2773.565430,...,0.333984,0.013672,-0.0,983.5000,19.6875,1015.6875,32.6875,350.0,0.0625,2
5875,2019-06-06,7297.6875,3106.4375,115063.9375,77039.1250,5384.512695,110655.0000,204620.2500,3.417985e+05,1315.011719,...,0.005859,-0.000000,-0.0,1494.2500,11.1875,1013.0000,44.8125,976.0,-0.0000,2
